In [1]:
def convert_to_datstring(dataframe_,columnName_='Date',format_= "%d/%m/%Y",ref_day_ ='01'):
    for i,dat in enumerate(dataframe_.loc[:,columnName_]):
        try: 
            dat = dat.strip()
            dataframe_.loc[i,'NewDate']=datetime.strptime(dat,format_)
            #print "zes"
        except:
             dat = dat.strip().split(r'/')
             dat[0]=ref_day_
             dat="/".join(dat)
             dataframe_.loc[i,'NewDate']=datetime.strptime(dat,format_)
    return dataframe_

In [6]:
import pandas as pd
from datetime import date, datetime, timedelta
import numpy as np
from collections import Counter
from __future__ import division
import matplotlib.pyplot as plt
emdat_ep=pd.read_csv('emdat_csv_formated.csv')
#con = (emdat_ep['Country'].tolist())
country_all = pd.read_csv('country.csv')
con = country_all.Country

emdat_ep_flood = emdat_ep[emdat_ep.Type.isin(['Flood'])]
emdat_ep_cholera = emdat_ep[emdat_ep.Name.isin(['Cholera'])]
#con = country_all.Country
#con=set(emdat_ep['Country'].tolist())
#count=0
appended_data =[]
for k in con:
    disease_country = emdat_ep_cholera[(emdat_ep_cholera.Country == k)]
    disease_flood = emdat_ep_flood[(emdat_ep_flood.Country == k)]   
#    count +=1
    #    print disease_flood
    week = timedelta(days = 15)
    timespan=np.arange(0,28)
    res_vec=np.zeros((timespan.shape[0],1))
    if disease_country.shape[0] == 0 or disease_flood.shape[0] == 0:
        for n in timespan:
#            res_vec[n,0] = 0
#            res_vec[n,0] = 0
            res_vec[n,0] = 0
    else:
        disease_country=disease_country.reset_index(drop=True)
        disease_country_dat = convert_to_datstring(dataframe_= disease_country)
        disease_flood=disease_flood.reset_index(drop=True)
        disease_flood_dat = convert_to_datstring(dataframe_= disease_flood)
        ################################################
        #print disease_country_dat
        df = pd.concat([disease_flood_dat, disease_country_dat], axis=1)
        df = df.drop(['Date','Type','Sub Type','Year','Name','Unnamed: 0','Unnamed: 0','Tot. Affected'], axis=1)
        df.columns = ['country_flood','flood_date','country_cholera','cholera_date']
        for n in timespan:
            borders= pd.concat([df.cholera_date - (n+1)*week, df.cholera_date - n*week], axis=1)
            borders.columns=['left_border','right_border']
    #        print borders
            counts = 0  
            total_occ_cholera=0
            for i, dis in enumerate(df.cholera_date):
                #cholera_date is sorted such that null values are at the end
                if pd.isnull(dis):
                    break
                else:
                    total_occ_cholera +=1
                for flood in df.flood_date:
                    left_b, right_b = borders.left_border[i],borders.right_border[i]
                    if flood < right_b and flood >= left_b:
                        counts += 1
                        break
#            res_vec[n,0] = counts
#            res_vec[n,0] = total_occ_cholera
            res_vec[n,0] = counts/total_occ_cholera
        data = pd.DataFrame(res_vec)
        appended_data.append(data)
appended_data = pd.concat(appended_data, axis=1)
appended_data = appended_data.T
appended_data.to_csv('data_calculated.csv')
country_delay_time = pd.read_csv('data_calculated.csv')
dfd = pd.concat([con, country_delay_time], axis=1)
#df = df.drop(['Unnamed: 0'], axis=1)
dfd.to_csv('data_calculated_new.csv')
#appended_data.to_csv('appedned.csv')

ValueError: labels ['Unnamed: 0'] not contained in axis

In [3]:
# country_delay_time = pd.read_csv('data_calculated.csv')
# dfd = pd.concat([country_all.Country, country_delay_time], axis=1)
# dfd
# country_delay_time
# # df_del_cholera = country_delay_time.drop(['Unnamed: 0','Unnamed: 1','Unnamed: 2','Unnamed: 4','Unnamed: 5','Unnamed: 7','Unnamed: 8','Unnamed: 10','Unnamed: 11','Unnamed: 13','Unnamed: 14','Unnamed: 16','Unnamed: 17','Unnamed: 19','Unnamed: 20','Unnamed: 22','Unnamed: 23','Unnamed: 25','Unnamed: 26','Unnamed: 28','Unnamed: 29','Unnamed: 31','Unnamed: 32','Unnamed: 34','Unnamed: 35','Unnamed: 37','Unnamed: 38','Unnamed: 40','Unnamed: 41','Unnamed: 43','Unnamed: 44','Unnamed: 46','Unnamed: 47','Unnamed: 49','Unnamed: 50','Unnamed: 52','Unnamed: 53','Unnamed: 55','Unnamed: 56','Unnamed: 58','Unnamed: 59','Unnamed: 61','Unnamed: 62','Unnamed: 64','Unnamed: 65','Unnamed: 67','Unnamed: 68','Unnamed: 70','Unnamed: 71','Unnamed: 73','Unnamed: 74','Unnamed: 76','Unnamed: 77','Unnamed: 79','Unnamed: 80','Unnamed: 82','Unnamed: 83','Unnamed: 85','Unnamed: 86','Unnamed: 88','Unnamed: 89'], axis=1)
# df_del_f = country_delay_time.drop(['Unnamed: 0','Unnamed: 1','Unnamed: 2','Unnamed: 4','Unnamed: 5','Unnamed: 7','Unnamed: 8','Unnamed: 10','Unnamed: 11','Unnamed: 13','Unnamed: 14','Unnamed: 16','Unnamed: 17','Unnamed: 19','Unnamed: 20','Unnamed: 22','Unnamed: 23','Unnamed: 25','Unnamed: 26','Unnamed: 28','Unnamed: 29','Unnamed: 31','Unnamed: 32','Unnamed: 34','Unnamed: 35','Unnamed: 37','Unnamed: 38','Unnamed: 40','Unnamed: 41','Unnamed: 43','Unnamed: 44','Unnamed: 46','Unnamed: 47','Unnamed: 49','Unnamed: 50','Unnamed: 52','Unnamed: 53','Unnamed: 55','Unnamed: 56','Unnamed: 58','Unnamed: 59','Unnamed: 61','Unnamed: 62','Unnamed: 64','Unnamed: 65','Unnamed: 67','Unnamed: 68','Unnamed: 70','Unnamed: 71','Unnamed: 73','Unnamed: 74','Unnamed: 76','Unnamed: 77','Unnamed: 79','Unnamed: 80','Unnamed: 82','Unnamed: 83','Unnamed: 85','Unnamed: 86','Unnamed: 88','Unnamed: 89'], axis=1)

# # df_del=df_del.T
# # df_del.to_csv('data_calculated.csv')